In [1]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import dataset

from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer



In [2]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


device = try_gpu(0)

# Preprocessing Text

In [3]:
import regex as re

with open('../data/ne_dedup.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'\s*[\u0964]\s*', r'\u0020\u0964\u0020', text)
    text = re.sub(r'\s*[\u003f]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*[\u002c]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*\n\s*','\n', text)
    #text = re.sub(r'\s*[\u0966-\u0976]+\s*','\u0020[\u0966-\u0976]\u0020', text)
    #text = re.sub(r'\s+?\s+', r'\u0020?\u0020', text)
    text = re.sub(r'[^\u0900-\u097F,?\s+]','', text)
    
    

In [4]:
c = re.sub(r'[^\u0900-\u097F,?]','','?नमस्तेnishant,')
#c = re.sub(r'\s*[\u002c]\s*','\u0020\u002c\u0020','?नमस्तेnishant,')
c


'?नमस्ते,'

In [5]:
hex(ord('०'))

'0x966'

In [6]:
len(text.split('\n'))

341961

In [7]:
train_iter = text.split('\n')[:30000]
test_iter = text.split('\n')[30000:35000]

In [8]:
train_iter[:100]

['बर्दिबास नगरपालिकाको तेस्रो नगर परिषदबाट पारित आव२०७३ । ७४ को संशोधित र २०७४ । ७५ को प्रस्तावित नीति ? कार्यक्रम तथा बजेट',
 'अार्थिक वर्ष २०७५७६ काे नदिजन्य पदार्थकाे उत्खनन् गरी बिक्रि वितरण तथा अान्तरिक निकासी गर्ने कार्यकाे बाेलपत्र सम्बन्धी सुचना',
 'सक्षार सप्तरी अभियानमा सप्तरीबासी सम्पूर्ण सरोकारवालाहरुको सहयोग र सहभागिताकाो लागि अनुराोध छ ।  सामुदायिक अध्ययन केन्द्रहरूको नविकरण सम्बन्धमा । ',
 'काठमाडौं ? १२ कातिक । राष्ट्रपति विद्यादेवी भण्डारी मित्रराष्ट्र कतारको चार दिवसीय औपचारिक भ्रमणमा आज त्यसतर्फ प्रस्थान गरेकी छन् । राष्ट्रपति विद्यादेवी भण्डारी कतारका अमिर शेख हमाद बीन खालिदा अल थानीको मैत्रीपूर्ण निमन्त्रणामा चार दिवसीय औपचारिक',
 'काठमाडौँ ? २६ कात्तिक । सरकारले सङ्घ ? प्रदेश र स्थानीय तहमा कर्मचारी समायोजन गर्नका लागि कर्मचारी समायोजन अध्यादेश२०७५ ल्याउने तयारी गरेको छ । सरकारले यसअघि ल्याएको',
 'काठमाडौं ? २६ कातिक । महानायक राजेश हमाल अहिले चलचित्र क्षेत्रमा पातलिए पनि उनको सिने जगतमा नामै काफी छ । कुनै समय बलिउड सुपरस्टार अमिताभ वच्चनसँग',
 'काठमाडौं ? २६ काति

In [9]:
print(len(train_iter),len(test_iter))

30000 5000


In [10]:
tokenizer = get_tokenizer(None)
vocab = build_vocab_from_iterator(
    map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [11]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [12]:
#device = torch.device('cpu')

In [13]:
#Train and Test Split

train_data = data_process(train_iter)
test_data = data_process(test_iter)

In [14]:
train_data[:100]

tensor([ 19020,   1025,    511,    761,  24762,    827, 103693,      1,   3726,
            32,  11495,      3,    792,      1,   1374,     32,   2702,    324,
             2,    107,     14,    330,  24336,     77,   5527,   8387,  28118,
        123255,  11037,     33,   6619,    480,     14,  34020,   3207,     13,
        176772,  44337,    640,   2987, 314958,   4492,   1852, 316978,    436,
         20999,    130,      3, 321145,      9, 151169,      4,      1,    950,
           305,  67904,  10488,    693,      1,     60,      2,    203,  27756,
             1,    387,   3505,   1589,  15993,   9954,    230,  11080,   1190,
          1417,     98,   6243,   3793,    783,      7,      1,    387,   3505,
          1589,  14473,  16876,   7713,  33868,  40783,  32454,   4344,  47676,
          7188,   9612,    230,  11080,   1190,    161,      2,    593,    302,
             1])

In [15]:
print(len(vocab))
print(len(train_iter),len(test_iter))
torch.cuda.empty_cache() 

torch.cuda.memory_allocated() 

342570
30000 5000


0

In [16]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [17]:
# batch_size = 35
# eval_batch_size = 10

# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# test_data = batchify(test_data, eval_batch_size)

In [18]:
#Sample Data


text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [19]:
sample_data

tensor([ 2086,  5694,   568,     0,   897, 28361,     0,   357,   465,   410,
         6548,   293,     0,   357,   465])

In [20]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[ 2086,   357],
        [ 5694,   465],
        [  568,   410],
        [    0,  6548],
        [  897,   293],
        [28361,     0],
        [    0,   357]], device='cuda:0')

In [21]:
train_data.shape

torch.Size([6406801])

In [22]:
bptt = 16
train_data = batchify(train_data, bptt)  # shape [seq_len, batch_size]
test_data = batchify(test_data, bptt)


import math
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

# Model Definition

In [23]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)





class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [24]:
# torch.cuda.memory_stats()


In [25]:
torch.cuda.memory_allocated()

60324352

In [26]:
#device = torch.device('cpu')

# Hyper-Parameter Tuning

In [27]:
ntokens = len(vocab)  # size of vocabulary
emsize = 300  # embedding dimension
d_hid = 800  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # number of heads in nn.MultiheadAttention
dropout = 0.05  # dropout probability
model = TransformerModel(ntokens, emsize,nhead, d_hid,
                         nlayers, dropout).to(device)

In [28]:
criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [29]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [30]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        #print(type(output))
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [31]:
softmax = nn.Softmax(dim=2)
softmax = nn.LogSoftmax(dim=2)

In [32]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            # print(output)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()

    return total_loss / (len(eval_data) - 1)


In [34]:
#            print('data')
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in data.t()]))
#             print(indices)
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in indices]))
#             print(len(targets))
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in target_indices]))

In [35]:
        # target_indices = targets.t()
        # print(indices.shape)
        # print(indices)
        # temp_gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0][i]
        # gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0]
        # print(temp_text)
        # print([[vocab.lookup_tokens(list(index))
        #        for index in indices][0][i]])
#         temp_text = [[vocab.lookup_tokens(list(index))]
#                       for index in indices[0][i]]
        # print(temp_text)
#         temp_text = [' '.join(temp_text)]
        # print(temp_text)
        # gen_data = vocab.lookup_tokens((list(gen_data))) + list(temp_gen_data)
        # gen_data = torch.tensor(
        #    (list(gen_data[0])+list(indices[0][i]))).unsqueeze(0)
        # gen_data = torch.concat([gen_data, torch.tensor(indices[0][i])], dim=1)
        #gen_data = ' '.join(gen_data)
        #return temp_txt
#         gen_data = data_process(temp_text)
#         gen_data = batchify(gen_data, 1)
        #return gen_data

In [36]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Training Data

In [37]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
import copy
best_val_loss = float('inf')
epochs = 8
best_model = None
#print(len(vocab.get_stoi()))
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    eval_loss = evaluate(model, test_data)
    eval_ppl = math.exp(eval_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         f'valid loss {eval_loss:5.2f} | valid ppl {eval_ppl:8.2f}')
    print('-' * 89)
    #best_model = copy.deepcopy(model)
    if eval_loss < best_val_loss:
       best_val_loss = eval_loss
       best_model = copy.deepcopy(model)

#     scheduler.step()
#save model
torch.save(best_model.state_dict(),'best_model_3bigx3.pt')


| epoch   1 |   200/25026 batches | lr 1.00 | ms/batch 167.88 | loss 10.69 | ppl 44002.80
| epoch   1 |   400/25026 batches | lr 1.00 | ms/batch 151.02 | loss  9.64 | ppl 15402.88
| epoch   1 |   600/25026 batches | lr 1.00 | ms/batch 151.48 | loss  9.36 | ppl 11595.82
| epoch   1 |   800/25026 batches | lr 1.00 | ms/batch 151.92 | loss  9.17 | ppl  9625.36
| epoch   1 |  1000/25026 batches | lr 1.00 | ms/batch 152.19 | loss  8.96 | ppl  7760.74
| epoch   1 |  1200/25026 batches | lr 1.00 | ms/batch 152.30 | loss  8.81 | ppl  6694.05
| epoch   1 |  1400/25026 batches | lr 1.00 | ms/batch 152.40 | loss  8.81 | ppl  6690.16
| epoch   1 |  1600/25026 batches | lr 1.00 | ms/batch 152.49 | loss  8.81 | ppl  6717.32
| epoch   1 |  1800/25026 batches | lr 1.00 | ms/batch 152.49 | loss  8.87 | ppl  7095.37
| epoch   1 |  2000/25026 batches | lr 1.00 | ms/batch 152.59 | loss  8.89 | ppl  7295.01
| epoch   1 |  2200/25026 batches | lr 1.00 | ms/batch 152.59 | loss  8.68 | ppl  5895.12
| epoch   

| epoch   1 | 18600/25026 batches | lr 1.00 | ms/batch 153.10 | loss  7.48 | ppl  1776.38
| epoch   1 | 18800/25026 batches | lr 1.00 | ms/batch 153.08 | loss  7.31 | ppl  1494.77
| epoch   1 | 19000/25026 batches | lr 1.00 | ms/batch 153.09 | loss  7.54 | ppl  1877.55
| epoch   1 | 19200/25026 batches | lr 1.00 | ms/batch 153.00 | loss  7.60 | ppl  2006.04
| epoch   1 | 19400/25026 batches | lr 1.00 | ms/batch 153.07 | loss  7.56 | ppl  1911.22
| epoch   1 | 19600/25026 batches | lr 1.00 | ms/batch 153.03 | loss  7.33 | ppl  1528.54
| epoch   1 | 19800/25026 batches | lr 1.00 | ms/batch 153.07 | loss  7.35 | ppl  1553.95
| epoch   1 | 20000/25026 batches | lr 1.00 | ms/batch 153.10 | loss  7.46 | ppl  1730.13
| epoch   1 | 20200/25026 batches | lr 1.00 | ms/batch 153.06 | loss  7.57 | ppl  1932.99
| epoch   1 | 20400/25026 batches | lr 1.00 | ms/batch 153.00 | loss  7.35 | ppl  1558.16
| epoch   1 | 20600/25026 batches | lr 1.00 | ms/batch 152.95 | loss  7.42 | ppl  1667.75
| epoch   

| epoch   2 | 11400/25026 batches | lr 1.00 | ms/batch 152.92 | loss  7.07 | ppl  1170.40
| epoch   2 | 11600/25026 batches | lr 1.00 | ms/batch 152.90 | loss  6.94 | ppl  1028.05
| epoch   2 | 11800/25026 batches | lr 1.00 | ms/batch 152.81 | loss  7.17 | ppl  1300.87
| epoch   2 | 12000/25026 batches | lr 1.00 | ms/batch 152.89 | loss  6.85 | ppl   944.62
| epoch   2 | 12200/25026 batches | lr 1.00 | ms/batch 152.79 | loss  6.88 | ppl   970.51
| epoch   2 | 12400/25026 batches | lr 1.00 | ms/batch 152.83 | loss  7.03 | ppl  1129.12
| epoch   2 | 12600/25026 batches | lr 1.00 | ms/batch 152.89 | loss  7.13 | ppl  1246.17
| epoch   2 | 12800/25026 batches | lr 1.00 | ms/batch 152.82 | loss  7.11 | ppl  1220.92
| epoch   2 | 13000/25026 batches | lr 1.00 | ms/batch 152.81 | loss  7.14 | ppl  1265.39
| epoch   2 | 13200/25026 batches | lr 1.00 | ms/batch 152.92 | loss  6.99 | ppl  1085.51
| epoch   2 | 13400/25026 batches | lr 1.00 | ms/batch 152.84 | loss  6.90 | ppl   988.65
| epoch   

| epoch   3 |  4200/25026 batches | lr 1.00 | ms/batch 152.79 | loss  6.86 | ppl   952.50
| epoch   3 |  4400/25026 batches | lr 1.00 | ms/batch 152.76 | loss  6.66 | ppl   777.90
| epoch   3 |  4600/25026 batches | lr 1.00 | ms/batch 152.78 | loss  6.63 | ppl   754.05
| epoch   3 |  4800/25026 batches | lr 1.00 | ms/batch 152.77 | loss  6.78 | ppl   878.62
| epoch   3 |  5000/25026 batches | lr 1.00 | ms/batch 152.66 | loss  6.70 | ppl   811.28
| epoch   3 |  5200/25026 batches | lr 1.00 | ms/batch 152.86 | loss  6.66 | ppl   780.31
| epoch   3 |  5400/25026 batches | lr 1.00 | ms/batch 152.83 | loss  6.69 | ppl   805.92
| epoch   3 |  5600/25026 batches | lr 1.00 | ms/batch 153.12 | loss  6.61 | ppl   745.80
| epoch   3 |  5800/25026 batches | lr 1.00 | ms/batch 153.11 | loss  6.60 | ppl   736.28
| epoch   3 |  6000/25026 batches | lr 1.00 | ms/batch 153.06 | loss  6.94 | ppl  1034.48
| epoch   3 |  6200/25026 batches | lr 1.00 | ms/batch 153.12 | loss  6.83 | ppl   924.70
| epoch   

| epoch   3 | 22600/25026 batches | lr 1.00 | ms/batch 153.24 | loss  6.64 | ppl   764.64
| epoch   3 | 22800/25026 batches | lr 1.00 | ms/batch 153.27 | loss  6.59 | ppl   728.13
| epoch   3 | 23000/25026 batches | lr 1.00 | ms/batch 153.50 | loss  6.56 | ppl   704.94
| epoch   3 | 23200/25026 batches | lr 1.00 | ms/batch 153.25 | loss  6.66 | ppl   784.18
| epoch   3 | 23400/25026 batches | lr 1.00 | ms/batch 153.22 | loss  6.33 | ppl   562.86
| epoch   3 | 23600/25026 batches | lr 1.00 | ms/batch 153.33 | loss  6.50 | ppl   666.52
| epoch   3 | 23800/25026 batches | lr 1.00 | ms/batch 153.21 | loss  6.61 | ppl   743.67
| epoch   3 | 24000/25026 batches | lr 1.00 | ms/batch 153.19 | loss  6.50 | ppl   668.29
| epoch   3 | 24200/25026 batches | lr 1.00 | ms/batch 153.18 | loss  6.28 | ppl   534.37
| epoch   3 | 24400/25026 batches | lr 1.00 | ms/batch 153.27 | loss  6.60 | ppl   738.66
| epoch   3 | 24600/25026 batches | lr 1.00 | ms/batch 153.18 | loss  6.69 | ppl   804.91
| epoch   

...

| epoch   6 |  1000/25026 batches | lr 1.00 | ms/batch 153.12 | loss  6.08 | ppl   435.38
| epoch   6 |  1200/25026 batches | lr 1.00 | ms/batch 153.16 | loss  5.93 | ppl   376.59
| epoch   6 |  1400/25026 batches | lr 1.00 | ms/batch 153.17 | loss  6.15 | ppl   467.54
| epoch   6 |  1600/25026 batches | lr 1.00 | ms/batch 153.12 | loss  6.31 | ppl   552.40
| epoch   6 |  1800/25026 batches | lr 1.00 | ms/batch 153.25 | loss  6.42 | ppl   615.73
| epoch   6 |  2000/25026 batches | lr 1.00 | ms/batch 153.21 | loss  6.54 | ppl   689.52
| epoch   6 |  2200/25026 batches | lr 1.00 | ms/batch 153.21 | loss  6.28 | ppl   532.71
| epoch   6 |  2400/25026 batches | lr 1.00 | ms/batch 153.21 | loss  6.36 | ppl   576.95
| epoch   6 |  2600/25026 batches | lr 1.00 | ms/batch 153.26 | loss  6.31 | ppl   550.54
| epoch   6 |  2800/25026 batches | lr 1.00 | ms/batch 153.49 | loss  6.08 | ppl   435.99
| epoch   6 |  3000/25026 batches | lr 1.00 | ms/batch 153.25 | loss  6.16 | ppl   473.77
| epoch   

| epoch   6 | 19400/25026 batches | lr 1.00 | ms/batch 153.35 | loss  6.11 | ppl   451.67
| epoch   6 | 19600/25026 batches | lr 1.00 | ms/batch 153.41 | loss  5.90 | ppl   363.86
| epoch   6 | 19800/25026 batches | lr 1.00 | ms/batch 153.44 | loss  5.95 | ppl   383.53
| epoch   6 | 20000/25026 batches | lr 1.00 | ms/batch 153.36 | loss  6.06 | ppl   426.32
| epoch   6 | 20200/25026 batches | lr 1.00 | ms/batch 153.41 | loss  6.18 | ppl   483.44
| epoch   6 | 20400/25026 batches | lr 1.00 | ms/batch 153.51 | loss  6.00 | ppl   403.32
| epoch   6 | 20600/25026 batches | lr 1.00 | ms/batch 153.42 | loss  6.07 | ppl   433.88
| epoch   6 | 20800/25026 batches | lr 1.00 | ms/batch 153.41 | loss  5.86 | ppl   350.68
| epoch   6 | 21000/25026 batches | lr 1.00 | ms/batch 153.44 | loss  6.08 | ppl   437.91
| epoch   6 | 21200/25026 batches | lr 1.00 | ms/batch 153.44 | loss  6.10 | ppl   447.19
| epoch   6 | 21400/25026 batches | lr 1.00 | ms/batch 153.47 | loss  5.92 | ppl   373.63
| epoch   

| epoch   7 | 12200/25026 batches | lr 1.00 | ms/batch 153.31 | loss  5.77 | ppl   322.09
| epoch   7 | 12400/25026 batches | lr 1.00 | ms/batch 153.25 | loss  5.92 | ppl   372.02
| epoch   7 | 12600/25026 batches | lr 1.00 | ms/batch 153.27 | loss  6.02 | ppl   411.66
| epoch   7 | 12800/25026 batches | lr 1.00 | ms/batch 153.24 | loss  5.99 | ppl   400.95
| epoch   7 | 13000/25026 batches | lr 1.00 | ms/batch 153.27 | loss  6.04 | ppl   418.00
| epoch   7 | 13200/25026 batches | lr 1.00 | ms/batch 153.24 | loss  5.85 | ppl   345.53
| epoch   7 | 13400/25026 batches | lr 1.00 | ms/batch 153.35 | loss  5.79 | ppl   328.23
| epoch   7 | 13600/25026 batches | lr 1.00 | ms/batch 153.23 | loss  5.94 | ppl   380.79
| epoch   7 | 13800/25026 batches | lr 1.00 | ms/batch 153.24 | loss  5.87 | ppl   354.09
| epoch   7 | 14000/25026 batches | lr 1.00 | ms/batch 153.20 | loss  5.84 | ppl   344.85
| epoch   7 | 14200/25026 batches | lr 1.00 | ms/batch 153.18 | loss  5.90 | ppl   365.94
| epoch   

| epoch   8 |  5000/25026 batches | lr 1.00 | ms/batch 153.31 | loss  5.74 | ppl   309.91
| epoch   8 |  5200/25026 batches | lr 1.00 | ms/batch 153.36 | loss  5.70 | ppl   298.07
| epoch   8 |  5400/25026 batches | lr 1.00 | ms/batch 153.31 | loss  5.75 | ppl   313.45
| epoch   8 |  5600/25026 batches | lr 1.00 | ms/batch 153.33 | loss  5.68 | ppl   292.05
| epoch   8 |  5800/25026 batches | lr 1.00 | ms/batch 153.31 | loss  5.63 | ppl   277.90
| epoch   8 |  6000/25026 batches | lr 1.00 | ms/batch 153.33 | loss  5.96 | ppl   386.45
| epoch   8 |  6200/25026 batches | lr 1.00 | ms/batch 153.26 | loss  5.85 | ppl   346.98
| epoch   8 |  6400/25026 batches | lr 1.00 | ms/batch 153.31 | loss  5.75 | ppl   313.43
| epoch   8 |  6600/25026 batches | lr 1.00 | ms/batch 153.29 | loss  5.83 | ppl   341.48
| epoch   8 |  6800/25026 batches | lr 1.00 | ms/batch 153.29 | loss  6.02 | ppl   410.39
| epoch   8 |  7000/25026 batches | lr 1.00 | ms/batch 153.24 | loss  5.80 | ppl   331.48
| epoch   

| epoch   8 | 23400/25026 batches | lr 1.00 | ms/batch 153.56 | loss  5.46 | ppl   235.22
| epoch   8 | 23600/25026 batches | lr 1.00 | ms/batch 153.44 | loss  5.62 | ppl   276.74
| epoch   8 | 23800/25026 batches | lr 1.00 | ms/batch 153.43 | loss  5.74 | ppl   312.39
| epoch   8 | 24000/25026 batches | lr 1.00 | ms/batch 153.44 | loss  5.66 | ppl   287.05
| epoch   8 | 24200/25026 batches | lr 1.00 | ms/batch 153.46 | loss  5.42 | ppl   226.68
| epoch   8 | 24400/25026 batches | lr 1.00 | ms/batch 153.67 | loss  5.73 | ppl   307.74
| epoch   8 | 24600/25026 batches | lr 1.00 | ms/batch 153.46 | loss  5.82 | ppl   337.36
| epoch   8 | 24800/25026 batches | lr 1.00 | ms/batch 153.49 | loss  5.93 | ppl   375.01
| epoch   8 | 25000/25026 batches | lr 1.00 | ms/batch 153.43 | loss  5.82 | ppl   336.92
-----------------------------------------------------------------------------------------
| end of epoch   8 | time: 4038.02s | valid loss  6.98 | valid ppl  1072.59
------------------------

# Data Generation

In [85]:
import time
def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        #print(indices[0],indices[1])
        #for j in range(batch_size):
        print('next word: ', [vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)
        pred_text.append([vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        if(batch_size < 16):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text



In [87]:
#Load saved model

# model.load_state_dict(torch.load('best_model_3bigx3.pt'))
# model.to(device)
# best_model = model

In [88]:

print(sample_data[:,-1].unsqueeze(1))
print(sample_data)
z = generator(best_model, sample_data[:,-1].unsqueeze(1),no_words = 50)

tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]], device='cuda:0')
tensor([[ 2086,   357],
        [ 5694,   465],
        [  568,   410],
        [    0,  6548],
        [  897,   293],
        [28361,     0],
        [    0,   357]], device='cuda:0')
i: 0
next word:  राज्य
0 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]], device='cuda:0') Pred_data:  tensor([[   53,   665,  5676, 13856,  1978,     3,   465]], device='cuda:0')
i: 1
next word:  अमेरिका
1 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357],
        [ 465]], device='cuda:0') Pred_data:  tensor([[   53,   665,  5676, 13856,  1978,     3,   465,   665]],
       device='cuda:0')
i: 2
next word:  र
2 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]

next word:  किम
34 Gen_data:  tensor([[ 2500],
        [  196],
        [ 4906],
        [ 9974],
        [13913],
        [    3],
        [  634],
        [ 2500],
        [  196],
        [ 4906],
        [ 9974],
        [13913],
        [    3],
        [  634],
        [ 2500],
        [  196]], device='cuda:0') Pred_data:  tensor([[  196,  4906,  9974, 13913,     3,   634,  2500,   196,  4906,  9974,
         13913,     3,   634,  2500,   196,  4906]], device='cuda:0')
i: 35
next word:  जोङ
35 Gen_data:  tensor([[  196],
        [ 4906],
        [ 9974],
        [13913],
        [    3],
        [  634],
        [ 2500],
        [  196],
        [ 4906],
        [ 9974],
        [13913],
        [    3],
        [  634],
        [ 2500],
        [  196],
        [ 4906]], device='cuda:0') Pred_data:  tensor([[    3,  9974, 13913,     3,  3968,  2500,   196,  4906,  9974, 13913,
             3,   634,  2500,   196,  4906,  9974]], device='cuda:0')
i: 36
next word:  उन
36 Gen_data

In [89]:
'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त ' +' '.join(z)

'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य अमेरिका र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ उन र उत्तर कोरियाली नेता किम जोङ'

In [90]:
evaluate(best_model, sample_data)

10.805072784423828

In [91]:
'''
आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , 
संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त

विकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय
'''

'\nआधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , \nसंयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त\n\nविकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय\n'

In [92]:
st = ['गर्ने']
st_i = data_process(st)

In [93]:
st_i = st_i.unsqueeze(1).to(device)

In [96]:
z_ = generator(best_model, st_i,no_words =50 )

i: 0
next word:  ?
0 Gen_data:  tensor([[13]], device='cuda:0') Pred_data:  tensor([[2]], device='cuda:0')
i: 1
next word:  सबै
1 Gen_data:  tensor([[13],
        [ 2]], device='cuda:0') Pred_data:  tensor([[ 2, 46]], device='cuda:0')
i: 2
next word:  ठाउँमा
2 Gen_data:  tensor([[13],
        [ 2],
        [46]], device='cuda:0') Pred_data:  tensor([[  2,  46, 333]], device='cuda:0')
i: 3
next word:  पहिरो
3 Gen_data:  tensor([[ 13],
        [  2],
        [ 46],
        [333]], device='cuda:0') Pred_data:  tensor([[   2,   46,  333, 1503]], device='cuda:0')
i: 4
next word:  जाँदा
4 Gen_data:  tensor([[  13],
        [   2],
        [  46],
        [ 333],
        [1503]], device='cuda:0') Pred_data:  tensor([[   2,   46,  333, 1503,  831]], device='cuda:0')
i: 5
next word:  पहिरो
5 Gen_data:  tensor([[  13],
        [   2],
        [  46],
        [ 333],
        [1503],
        [ 831]], device='cuda:0') Pred_data:  tensor([[   2,   46,  333, 1503,  831, 1503]], device='cuda:0')
i: 6


i: 37
next word:  सकेनौं
37 Gen_data:  tensor([[ 268],
        [ 788],
        [4567],
        [ 849],
        [  12],
        [   2],
        [ 102],
        [  26],
        [ 136],
        [ 325],
        [   4],
        [   1],
        [  15],
        [  26],
        [ 143],
        [  10]], device='cuda:0') Pred_data:  tensor([[  100, 17643,  4024,    65,     2,   143,    15,   136,  2866,     4,
             1,    19,    26,   143,    10, 10039]], device='cuda:0')
i: 38
next word:  ।
38 Gen_data:  tensor([[  788],
        [ 4567],
        [  849],
        [   12],
        [    2],
        [  102],
        [   26],
        [  136],
        [  325],
        [    4],
        [    1],
        [   15],
        [   26],
        [  143],
        [   10],
        [10039]], device='cuda:0') Pred_data:  tensor([[17643,  4024,    65,     2,   143,    15,   136,  2866,     4,     1,
            19,    26,   143,    10, 10039,     1]], device='cuda:0')
i: 39
next word:  तर
39 Gen_data:  tensor

In [97]:
' '.join(z_)

'? सबै ठाउँमा पहिरो जाँदा पहिरो जाने र पहिरो जाने क्रम बढेपछि बाटो अवरुद्ध भएको छ । नारायणगढमुग्लिन सडक आयोजनाका प्रमुख एवं प्रवक्ता सुरेश अधिकारीले भने ? अब काम अघि बढेको छ । यो काम हामीले गर्न सकेनौं । तर ? हामीले काम गर्न सकेनौं । हामीले हाम्रो शिक्षा ?'